# Get Fundamentals Data. 

In [1]:
# Python module. 
import re, os  
import pandas as pd

# Change the current directory from (./notebook) to root directory. 
while not re.match(r".+MADS-CAP$", os.getcwd()): 
	os.chdir("..") 
	
print(f"Current directory: ({os.getcwd()})") 

# For clearing safe warnings. Not important. 
from IPython.display import clear_output

# Custom modules. 
from source.modules.processor_features import compile_features_each_ticker 
from source.modules.processor_fundamental import (
	get_snp_wikiinfo, get_corporate_overview_alphav, get_fundamental_alphav, 
	compute_extra_fundamental, score_fundamental, rank_fundamental, 
	within_sector_diff, rank_within_sector_diff, 
) 

# Custom configs. 
from source.config_py.config import (
	DIR_DATASET_FUNDAMENTAL, DIR_DATASET_UTIL, 
	TICKER_FROM_SNP, TICKER_TO_COLLECT, TICKER_TO_EXCLUDE, 
	FINANCIAL_EVAL_QUALITY, 
)

Current directory: (/Users/lioneltay/Dropbox/Courses/michigan_mads/SIADS_697_/submission/MADS-CAP)


/Users/lioneltay/.local/share/virtualenvs/MADS-CAP-gLyVeGLS/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configurations (general). 

In [2]:
# Pandas DF config. 
pd.set_option("display.max_rows", 50, "display.max_columns", 100, "display.max_colwidth", 50)

# List of ticker to collect data. 
snp = pd.read_csv(f"{DIR_DATASET_UTIL}/snp500_tickers.csv").Symbol if TICKER_FROM_SNP else set() 
ticker_to_collect = TICKER_TO_COLLECT.union(snp).difference(TICKER_TO_EXCLUDE) 

# Whether to load the cache file for the fundamental data. 
load_cache = True 

# For clearing the output. Not important. 
clear_output() 

## Get sector info. 

In [3]:
df_snp_info = get_snp_wikiinfo() 

# Preview. 
df_snp_info 

Getting S&P sector info from (Wikipedia).


,ticker,security,sec_filings,gics_sector,gics_sub_industry,headquarters_location,date_first_added,cik,founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


### Cache dataset. 

In [4]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_UTIL, "snp500_sector_info.csv") 
# df_snp_info.to_csv(filepath, index=False) 

## Get corporate info. 

In [5]:
# # Assign (load_cache) to (False) to overwrite existing data for all tickers. 
# # Otherwise, it will append unavailable data to the existing dataset. 
# # Take about 15 minutes to complete the entire S&P tickers. 

# filepath = os.path.join(DIR_DATASET_UTIL, "corporate_overview.csv") 
# df_corporate = compile_features_each_ticker(get_corporate_overview_alphav, filepath, ticker_to_collect, load_cache=load_cache) 

# # Preview. 
# df_corporate 

## Get fundamental data. 

In [6]:
# Assign (load_cache) to (False) to overwrite existing data for all tickers. 
# Otherwise, it will append unavailable data to the existing dataset. 
# Take about 30 minutes to complete the entire S&P tickers. 

filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement.csv") 
df_finstate = compile_features_each_ticker(get_fundamental_alphav, filepath, ticker_to_collect, load_cache=load_cache) 

# Preview. 
df_finstate 

,ticker,fiscalDateEnding,reportedCurrency,accumulatedDepreciationAmortizationPPE,capitalExpenditures,capitalLeaseObligations,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,cashflowFromFinancing,cashflowFromInvestment,changeInCashAndCashEquivalents,changeInExchangeRate,changeInInventory,changeInOperatingAssets,changeInOperatingLiabilities,changeInReceivables,commonStock,commonStockSharesOutstanding,comprehensiveIncomeNetOfTax,costOfRevenue,costofGoodsAndServicesSold,currentAccountsPayable,currentDebt,currentLongTermDebt,currentNetReceivables,deferredRevenue,depreciation,depreciationAndAmortization,depreciationDepletionAndAmortization,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,ebit,ebitda,goodwill,grossProfit,incomeBeforeTax,incomeTaxExpense,intangibleAssets,intangibleAssetsExcludingGoodwill,interestAndDebtExpense,interestExpense,interestIncome,inventory,investmentIncomeNet,investments,longTermDebt,longTermDebtNoncurrent,longTermInvestments,netIncomeFromContinuingOperations,netIncome_x,netIncome_y,netInterestIncome,nonInterestIncome,operatingCashflow,operatingExpenses,operatingIncome,otherCurrentAssets,otherCurrentLiabilities,otherNonCurrentLiabilities,otherNonCurrrentAssets,otherNonOperatingIncome,paymentsForOperatingActivities,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromOperatingActivities,proceedsFromRepaymentsOfShortTermDebt,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,profitLoss,propertyPlantEquipment,researchAndDevelopment,retainedEarnings,sellingGeneralAndAdministrative,shortLongTermDebtTotal,shortTermDebt,shortTermInvestments,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiabilities,totalNonCurrentAssets,totalNonCurrentLiabilities,totalRevenue,totalShareholderEquity,treasuryStock
0,AAPL,30/9/21,USD,7.028300e+10,1.108500e+10,769000000.0,3.494000e+10,6.263900e+10,-9.335300e+10,-1.454500e+10,-3.860000e+09,NaN,2.642000e+09,2.471200e+10,1.980100e+10,1.402800e+10,5.736500e+10,1.642679e+10,9.524900e+10,2.349540e+11,2.129810e+11,5.476300e+10,1.569200e+10,9.613000e+09,5.150600e+10,1.951200e+10,9.500000e+09,1.128400e+10,11284000000,1.446700e+10,1.446700e+10,NaN,1.118520e+11,1.231360e+11,NaN,1.528360e+11,1.092070e+11,1.452700e+10,NaN,NaN,2.645000e+09,2.645000e+09,2.843000e+09,6.580000e+09,2.843000e+09,3.189940e+11,1.187000e+11,1.091060e+11,1.278770e+11,9.468000e+10,94680000000,94680000000,-2.645000e+09,3.658170e+11,1.040380e+11,4.388700e+10,1.089490e+11,1.411100e+10,4.749300e+10,5.332500e+10,4.884900e+10,60000000.0,4.087000e+09,8.597100e+10,8.597100e+10,NaN,1.105000e+09,2.039300e+10,NaN,NaN,2.044000e+09,-8.486600e+10,NaN,9.468000e+10,3.944000e+10,2.191400e+10,5.562000e+09,2.197300e+10,2.417830e+11,6.000000e+09,2.769900e+10,3.510020e+11,1.348360e+11,1.254810e+11,2.879120e+11,2.161660e+11,1.624310e+11,3.631720e+11,6.309000e+10,NaN
1,AAPL,30/9/20,USD,6.676000e+10,7.309000e+09,637000000.0,3.801600e+10,9.094300e+10,-8.682000e+10,-4.289000e+09,-1.043500e+10,NaN,1.270000e+08,1.245000e+09,6.935000e+09,-8.470000e+09,5.077900e+10,1.697676e+10,5.745300e+10,1.894750e+11,1.695590e+11,4.229600e+10,1.379300e+10,8.773000e+09,3.744500e+10,1.684300e+10,9.700000e+09,1.105600e+10,11056000000,1.408100e+10,1.408100e+10,NaN,6.996400e+10,8.102000e+10,NaN,1.049560e+11,6.709100e+10,9.680000e+09,NaN,NaN,2.873000e+09,2.873000e+09,3.763000e+09,4.061000e+09,3.763000e+09,3.456440e+11,1.258730e+11,9.866700e+10,1.008870e+11,5.741100e+10,57411000000,57411000000,-2.873000e+09,2.745150e+11,8.067400e+10,3.866800e+10,6.628800e+10,1.126400e+10,4.268400e+10,5.449000e+10,4.252200e+10,-87000000.0,4.502000e+09,7.235800e+10,7.235800e+10,NaN,8.800000e+08,1.609100e+10,NaN,NaN,-1.926000e+09,-7.147800e+10,NaN,5.741100e+10,3.676600e+10,1.875200e+10,1.496600e+10,1.991600e+10,2.168600e+11,4.996000e+09,5.29270

## Compute additional fundamental data. 

In [7]:
# Compute change / growth rate, arithmetic, and ratio to create additional fundamental data.
df_finstate_ext = compute_extra_fundamental(df_finstate) 

# Preview. 
df_finstate_ext 

Compute additional fundamental data.


,ticker,fiscalDateEnding,totalRevenue,researchAndDevelopment,capitalExpenditures,depreciationAndAmortization,grossProfit,interestExpense,ebit,incomeBeforeTax,netIncomeFromContinuingOperations,netIncome_x,inventory,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiabilities,longTermDebtNoncurrent,totalShareholderEquity,operatingCashflow,growth_totalRevenue,growth_grossProfit,growth_netIncomeFromOperations,growth_ebit,growth_netIncome_x,growth_operatingCashflow,ownerEarnings,assetsQuick,workingCapital,shareholderEquity,turnover_asset,turnover_operatingCashFlow,margin_grossProfit,margin_netIncomeFromOperations,workingCapitalTurnover,inventoryRatio,capexRatio,researchAndDevelopmentRatio,incomeNetPretaxPerRevenue,currentRatio,quickRatio,cashRatio,longTermDebtRatio,cashFlowOperatingRatio,interestPayRatio,debtEquityRatio,shareholderEquityRatio,returnEquityRatio,cashFlowOperatingPerShare
0,AAPL,30/9/21,3.631720e+11,2.191400e+10,1.108500e+10,1.128400e+10,1.528360e+11,2.645000e+09,1.118520e+11,1.092070e+11,9.468000e+10,94680000000,6.580000e+09,3.510020e+11,1.348360e+11,1.254810e+11,2.879120e+11,1.091060e+11,6.309000e+10,1.040380e+11,0.336951,0.456191,0.649161,0.598708,0.649161,0.289610,9.487900e+10,1.282560e+11,9.355000e+09,1.261800e+11,1.034672,0.286470,0.420836,0.260703,0.025759,0.018118,0.030523,0.060341,0.300703,1.074553,1.022115,0.829114,0.999075,0.829114,0.017306,4.563512,0.359485,1.503867,1.649041
1,AAPL,30/9/20,2.716420e+11,1.875200e+10,7.309000e+09,1.105600e+10,1.049560e+11,2.873000e+09,6.996400e+10,6.709100e+10,5.741100e+10,57411000000,4.061000e+09,3.238880e+11,1.437130e+11,1.053920e+11,2.585490e+11,9.866700e+10,6.533900e+10,8.067400e+10,0.058629,0.066713,0.039000,0.009392,0.039000,0.162600,6.115800e+10,1.396520e+11,3.832100e+10,1.306780e+11,0.838691,0.296986,0.386376,0.211348,0.141072,0.014950,0.026907,0.069032,0.246983,1.363604,1.325072,0.765466,1.470644,0.765466,0.027373,3.957039,0.403467,0.936011,1.234699
2,AAPL,30/9/19,2.565980e+11,1.621700e+10,1.049500e+10,1.254700e+10,9.839200e+10,3.576000e+09,6.931300e+10,6.573700e+10,5.525600e+10,55256000000,4.106000e+09,3.385160e+11,1.628190e+11,1.057180e+11,2.480280e+11,9.180700e+10,9.048800e+10,6.939100e+10,-0.033875,-0.033848,-0.071811,-0.089700,-0.071811,-0.103869,5.730800e+10,1.587130e+11,5.710100e+10,1.809760e+11,0.758008,0.270427,0.383448,0.215341,0.222531,0.016002,0.040901,0.063200,0.256187,1.540126,1.501286,0.656378,1.396580,0.656378,0.036344,2.741004,0.534616,0.633322,0.766853
3,AAPL,30/9/18,2.655950e+11,1.423600e+10,1.331300e+10,1.090300e+10,1.018390e+11,3.240000e+09,7.614300e+10,7.290300e+10,5.953100e+10,59531000000,3.956000e+09,3.657250e+11,1.313390e+11,1.159290e+11,2.585780e+11,9.373500e+10,1.071470e+11,7.743400e+10,0.158620,0.154820,0.231226,0.146525,0.231226,0.205668,5.712100e+10,1.273830e+11,1.541000e+10,2.142940e+11,0.726215,0.291549,0.383437,0.224142,0.058021,0.014895,0.050125,0.053600,0.274489,1.132926,1.098802,0.667943,1.285750,0.667943,0.031815,2.413301,0.585943,0.533109,0.722689
4,AAPL,30/9/17,2.292340e+11,1.158100e+10,1.279500e+10,1.200000e+09,8.818600e+10,2.323000e+09,6.641200e+10,6.408900e+10,4.835100e+10,48351000000,4.855000e+09,3.753190e+11,1.286450e+11,1.008140e+11,2.412720e+11,9.720700e+10,1.340470e+11,6.422500e+10,NaN,NaN,NaN,NaN,NaN,NaN,3.675600e+10,1.237900e+11,2.783100e+10,2.680940e+11,0.610771,0.280172,0.384699,0.210924,0.121409,0.021179,0.055816,0.050520,0.279579,1.276063,1.227905,0.637064,1.516750,0.637064,0.026342,1.799906,0.714310,0.274202,0.479123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,SJM,30/4/22,8.066900e+09,NaN,4.175000e+08,2.296000e+08,2.700700e+09,1.000000e+05,1.023800e+09,8.438000e+08,6.317000e+08,631700000,1.089300e+09,1.605500e+10,2.010100e+09,1.952800e+09,7.914900e+09,4.310600e+09,8.140100e+09,1.136300e+09,-0.003077,-0.139548,-0.279128,-0.261754,-0.279128,-0.27

### Cache dataset. 

In [8]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_ext.csv") 
# df_finstate_ext.to_csv(filepath) 

## Rank corporate fundamental. 

### Rank by score. 

In [9]:
# Rank the fundamental quality. 
df_rank_fundamental = rank_fundamental(score_fundamental, df_finstate_ext) 

# Preview 
df_rank_fundamental 

Ranking the fundamental quality.


,ticker,score_growth_totalRevenue,score_growth_grossProfit,score_growth_netIncomeFromOperations,score_growth_ebit,score_growth_netIncome_x,score_growth_operatingCashflow,score_turnover_asset,score_turnover_operatingCashFlow,score_margin_grossProfit,score_margin_netIncomeFromOperations,total_score,rank_quantl
0,ZTS,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,1,8.0,0.049603
1,ORLY,1.0,1.0,1.0,1.0,1.0,1.0,1,0,1,0,8.0,0.049603
2,CNC,1.0,1.0,1.0,1.0,1.0,1.0,1,0,1,0,8.0,0.049603
3,OTIS,1.0,1.0,1.0,1.0,1.0,1.0,1,0,1,0,8.0,0.049603
4,GOOG,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,1,8.0,0.049603
...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,DAL,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987103
500,RCL,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987103
501,APTV,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987103
502,CCL,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987103


### Cache dataset. 

In [10]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_rankbyscore.csv") 
# df_rank_fundamental.to_csv(filepath) 

### Rank by within sector performance difference. 

In [11]:
# Rank the performance difference within sector. 
usecols = ["ticker", "fiscalDateEnding", "gics_sector"] + list(FINANCIAL_EVAL_QUALITY.keys()) 
df_within_sector_diff = within_sector_diff(df_finstate_ext, df_snp_info, usecols, yearspec=("fiscalDateEnding", 2018), groupcols=["gics_sector"]) 
df_rank_within_sector_diff = rank_within_sector_diff(df_within_sector_diff, groupcols=["gics_sector"]) 

# Preview. 
df_rank_within_sector_diff 

Ranking the performance difference within sector.


,fiscalDateEnding,gics_sector,ticker,growth_totalRevenue,growth_grossProfit,growth_netIncomeFromOperations,growth_ebit,growth_netIncome_x,growth_operatingCashflow,turnover_asset,turnover_operatingCashFlow,margin_grossProfit,margin_netIncomeFromOperations,rank_quantl
0,2018-09-30,Information Technology,AAPL,0.027334,0.008354,-1.734904,-38.190821,-0.557229,-0.056065,0.038020,0.027140,-0.140215,0.061013,0.479452
1,2018-12-31,Consumer Discretionary,AMZN,0.190835,-0.551053,2.083607,1.861975,2.450725,1.231377,0.207272,-0.019237,-0.145049,-0.062173,0.701754
2,2018-12-31,Industrials,CSX,-0.028211,-0.663287,-0.850193,0.067080,-0.788426,0.111576,-0.614985,0.235881,0.257722,0.178470,0.718310
3,2018-05-31,Information Technology,PAYX,-0.060020,-1.774177,-1.823709,-38.307359,-0.646035,0.067297,-0.261470,0.113481,0.174900,0.113302,0.301370
4,2018-12-31,Industrials,NLSN,-0.180917,-2.085795,-3.045927,-1.672963,-3.052925,-0.417484,-0.533236,0.014763,-0.550351,-0.217486,0.014085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,2018-12-31,Industrials,LUV,-0.072049,-0.913321,-0.748310,-0.278007,-0.658965,0.020237,-0.098319,0.064169,-0.146720,0.009436,0.197183
487,2018-12-31,Information Technology,MPWR,0.105381,0.102814,-1.351664,-37.871547,-0.173990,-0.204716,0.045809,-0.021526,0.030396,0.017626,0.671233
488,2018-12-31,Financials,IVZ,-0.031030,-0.481280,0.046700,-0.276917,-0.276540,-0.145963,-0.031265,-0.198015,0.309315,-0.005656,0.107692
489,2018-05-31,Industrials,FDX,-0.024489,-0.589325,0.070507,-0.288504,0.132274,-0.277045,0.315412,-0.087181,-0.143450,-0.032933,0.436620


### Cache dataset. 

In [12]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_rankbywithinsector.csv") 
# df_rank_within_sector_diff.to_csv(filepath) 

## Output top N ticker ranks. 

### By Score. 

In [13]:
df_rank_fundamental \
	.loc[df_rank_fundamental["rank_quantl"] <= 0.05, "ticker"] \
	.unique() 

array(['ZTS', 'ORLY', 'CNC', 'OTIS', 'GOOG', 'PFE', 'POOL', 'CE', 'CDNS',
       'REGN', 'CBRE', 'SBNY', 'SIVB', 'TER', 'TFC', 'ARE', 'ODFL',
       'CPRT', 'NVDA', 'MU', 'GRMN', 'ICE', 'IDXX', 'INTU', 'IP', 'ISRG',
       'EW', 'TROW', 'KLAC', 'LRCX', 'MA', 'META', 'DPZ', 'MSFT', 'DHI',
       'DG', 'EPAM', 'TSCO', 'CMG', 'GOOGL', 'ALGN', 'ULTA', 'AMZN',
       'ADBE', 'UPS', 'AMAT', 'VRTX', 'ANET', 'AAPL'], dtype=object)

### By within sector performance difference. 

In [14]:
df_rank_within_sector_diff \
	.loc[df_rank_within_sector_diff["rank_quantl"] <= 0.05, "ticker"] \
	.unique() 

array(['NLSN', 'XRAY', 'RE', 'ZBH', 'MAA', 'NWL', 'QCOM', 'WHR', 'NRG',
       'AIG', 'AFL', 'CSCO', 'KHC', 'AAL', 'FIS', 'CVS', 'BKR', 'NEM',
       'ALK'], dtype=object)